In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from encode_data import *
from midi_data import *

In [3]:
from tqdm import tqdm

## Encode music21 stream to text representation and back

### Load midi data

In [4]:
path = Path('data/midi')
csv_path = path/'metadata'

In [5]:
source_dir = 'midi_transform_v1'
out_dir = 'midi_transcribe_v1'
source_csv = csv_path/f'{source_dir}.csv'
out_csv = csv_path/f'{out_dir}.csv'

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv(source_csv, index_col=0); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (12,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,artist,bpm,genres,ht_bpm,ht_key,ht_mode,ht_offset,ht_time_signature,inferred_key,inferred_offset,...,midi_title,original_path,parts,seconds,section,song_url,source,time_signature,title,midi_transform_v1
0,wayne-sharpe,128.0,NaN,128.0,C,1.0,0.0,4.0,C major,0.0,...,yu-gi-oh3,data/midi/midi_sources/hooktheory/pianoroll/w/...,"intro,chorus",15.000000,chorus,https://www.hooktheory.com/theorytab/view/wayn...,hooktheory,4/4,yu-gi-oh-theme-song,data/midi/midi_transform_v1/classic_piano/chpn...
0,Tune Up,142.0,dance,NaN,NaN,NaN,NaN,NaN,F major,-5.0,...,NaN,NaN,NaN,28.732394,NaN,NaN,freemidi,NaN,Bounce,data/midi/midi_transform_v1/classic_piano/chpn...
0,Kona,176.0,"pop,inferred",NaN,NaN,NaN,NaN,NaN,B minor,-2.0,...,NaN,NaN,NaN,1.193182,NaN,NaN,midiworld,4/4,Drumloop,data/midi/midi_transform_v1/classic_piano/chpn...
0,,117.0,classical,NaN,NaN,NaN,NaN,NaN,A minor,0.0,...,NaN,NaN,NaN,212.820513,NaN,NaN,ecomp,4/4,II. Intermezzo in A Minor,data/midi/midi_transform_v1/classic_piano/chpn...
0,Alex Gaudino,143.0,"EDM,inferred",NaN,NaN,NaN,NaN,NaN,A- major,4.0,...,NaN,NaN,NaN,28.531469,NaN,NaN,cprato,NaN,Calabria (Drunken Monkey Remix) (Midi By Carlo...,data/midi/midi_transform_v1/classic_piano/chpn...


In [8]:
df_filtered = df.loc[df['time_signature'] == '4/4']; df_filtered.shape

(30201, 22)

In [9]:
from data_sources import process_parallel

In [14]:
def transcribe_file(idxrow):
    idx,row = idxrow
    midi_file = row[source_dir]
    if not isinstance(midi_file, str) or not Path(midi_file).exists(): return idx,None
    out_file = Path(midi_file.replace(f'/{source_dir}/', f'/{out_dir}/')).with_suffix('.txt')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): return idx,str(out_file)
    try:
        seq = midi2seq(midi_file)
        string_repr = seq2str(seq)
        with open(out_file, 'w') as tf:
            tf.write(string_repr)
    except Exception as e:
        print('Error converting midi to sequence')
        return idx,None
    return idx,str(out_file)

In [16]:
# for r in df_filtered.iterrows():
#     transcribe_file(r)

In [ ]:
process_parallel(transcribe_file, df_filtered.iterrows(), total=df_filtered.shape[0])

Error converting midi to sequence
Error converting midi to sequence


In [51]:
tdf = pd.DataFrame(data={out_dir: list(transcribed_files.values())}, index=transcribed_files.keys())

In [52]:
merged_df = df.join(tdf, how='outer'); tdf.shape, df.shape, merged_df.shape

In [56]:
merged_df.to_csv(out_csv); merged_df.head()

,parts,ht_mode,ht_key,original_path,genres,seconds,section,midi_keyc,time_signature,ht_time_signature,ht_bpm,instruments,bpm,midi_title,song_url,artist,inferred_key,inferred_keyc,transcribed_keyc_v1
0,"['intro', 'chorus']",1.0,C,../data/midi/hooktheory/pianoroll/w/wayne-shar...,[],15.000000,chorus,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,128,"['Piano', 'Piano']",128.0,yu-gi-oh3,https://www.hooktheory.com/theorytab/view/wayn...,wayne-sharpe,C major,C major,../data/midi/transcribedc_v1/hooktheory/pianor...
1,"['intro', 'chorus']",1.0,C,../data/midi/hooktheory/pianoroll/w/wayne-shar...,[],25.411765,intro,../data/midi/transposed/hooktheory/pianoroll/w...,3/4,3,85,['Piano'],85.0,yu-gi-oh,https://www.hooktheory.com/theorytab/view/wayn...,wayne-sharpe,C major,C major,NaN
2,['chorus'],6.0,D,../data/midi/hooktheory/pianoroll/w/what-a-day...,['Jazz'],10.000000,chorus,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,96,"['Piano', 'Piano']",96.0,kiefer,https://www.hooktheory.com/theorytab/view/what...,what-a-day,A minor,A minor,../data/midi/transcribedc_v1/hooktheory/pianor...
3,"['verse', 'pre-chorus', 'chorus']",6.0,D,../data/midi/hooktheory/pianoroll/w/whiteflame...,"['J-Pop', 'Pop']",25.263158,chorus,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,152,"['Piano', 'Piano']",152.0,Senbonzakura,https://www.hooktheory.com/theorytab/view/whit...,whiteflame,D minor,D minor,../data/midi/transcribedc_v1/hooktheory/pianor...
4,"['verse', 'pre-chorus', 'chorus']",6.0,D,../data/midi/hooktheory/pianoroll/w/whiteflame...,"['J-Pop', 'Pop']",12.631579,verse,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,152,"['Piano', 'Piano']",152.0,Senbonzakura,https://www.hooktheory.com/theorytab/view/whit...,whiteflame,D minor,D minor,../data/midi/transcribedc_v1/hooktheory/pianor...
